In [ ]:
import os
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
import matplotlib.pyplot as plt
import numpy as np
from Bio.Seq import Seq
from collections import defaultdict
import re
import logging
from Bio.SeqUtils import GC, GC123
from Bio.Alphabet import generic_dna
from collections import Counter
from Bio.Seq import MutableSeq
import pandas as pd
from scipy import stats
from scipy import mean as sc_m
import itertools
import pandas as pd
import researchpy as rp
from scipy import stats
import seaborn as sns
import statsmodels.api as sm
from statsmodels.formula.api import ols
import matplotlib.pyplot as plt
from statsmodels.stats import anova 
from matplotlib import cm #https://matplotlib.org/3.1.1/tutorials/colors/colormaps.html
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.libqsturng import psturng
from tabulate import tabulate

%matplotlib inline

In [ ]:
def medias_df_gc(df_gc):
    
    df_medias = pd.DataFrame()
    
    for virus_type in df_gc["Tipo"].unique():
        
        df_tipo = df_gc.loc[df_gc["Tipo"].isin([virus_type])]
        
        if len(df_tipo) > 1:
            
            df_prov = pd.DataFrame()
            
            col_gen = df_tipo["Genero"].unique()
            
            col_type = [virus_type]
            
            col_gc_total = np.mean(df_tipo["Total_GC"])
            col_gc_12 = np.mean(df_tipo["GC12"])
            col_gc_3 = np.mean(df_tipo["GC3"])

            col_gc_total_desv = np.std(df_tipo["Total_GC"])
            col_gc_12_desv = np.std(df_tipo["GC12"])
            col_gc_3_desv = np.std(df_tipo["GC3"])

            df_prov["Genero"] = col_gen
            df_prov["Tipo"] = col_type
            df_prov["Total_GC"] = col_gc_total
            df_prov["GC12"] = col_gc_12
            df_prov["GC3"] = col_gc_3
            df_prov["Total_GC_desv"] = col_gc_total_desv
            df_prov["GC12_desv"] = col_gc_12_desv
            df_prov["GC3_desv"] = col_gc_3_desv
            
            df_medias = pd.concat([df_medias, df_prov], axis = 0, sort = False)
            
        else:
            
            df_medias = pd.concat([df_medias, df_tipo], axis = 0, sort = False)
            
    return df_medias

In [ ]:
def medias_df_gc_genes(df_gc_genes):
    
    df_medias = pd.DataFrame()
    
    for virus_type in df_gc_genes["Tipo"].unique():
        
        df_tipo = df_gc_genes.loc[df_gc_genes["Tipo"].isin([virus_type])]
        
        for gene in df_tipo["Gen"].unique():
            
            df_gen = df_tipo.loc[df_tipo["Gen"].isin([gene])]
        
            if len(df_gen) > 1:
            
                df_prov = pd.DataFrame()
            
                col_genero = df_gen["Genero"].unique()
                col_type = [virus_type]
                col_gen = [gene]
            
                col_gc_total = np.mean(df_gen["Total_GC"])
                col_gc_12 = np.mean(df_gen["GC12"])
                col_gc_3 = np.mean(df_gen["GC3"])

                col_gc_total_desv = np.std(df_gen["Total_GC"])
                col_gc_12_desv = np.std(df_gen["GC12"])
                col_gc_3_desv = np.std(df_gen["GC3"])

                df_prov["Genero"] = col_genero
                df_prov["Tipo"] = col_type
                df_prov["Gen"] = col_gen
                df_prov["Total_GC"] = col_gc_total
                df_prov["GC12"] = col_gc_12
                df_prov["GC3"] = col_gc_3
                df_prov["Total_GC_desv"] = col_gc_total_desv
                df_prov["GC12_desv"] = col_gc_12_desv
                df_prov["GC3_desv"] = col_gc_3_desv
            
                df_medias = pd.concat([df_medias, df_prov], axis = 0, sort = False)
            
            else:
            
                df_medias = pd.concat([df_medias, df_gen], axis = 0, sort = False)
            
    return df_medias

In [ ]:
def plot_neutrality(df_genome, output_file):
    fig_size = plt.rcParams["figure.figsize"]
    fig_size[0] = 30
    fig_size[1] = 15
    plt.rcParams["figure.figsize"] = fig_size
    plt.subplots_adjust(wspace = 0.3)

    number = 1

    #plt.suptitle("Genoma concatenado. Media de las secuencias por tipo", fontsize = 35)

    for virus_genus in df_genome["Genero"].unique():
    
        gc3 = df_genome.loc[df_genome["Genero"].isin([virus_genus])]["GC3"]
        gc12 = df_genome.loc[df_genome["Genero"].isin([virus_genus])]["GC12"]
        
        slope, intercept, rvalue, pvalue, stderr = stats.linregress(gc3, gc12)

        plt.subplot(1,3, number)
        plt.plot(gc3, gc12, "o", label = "GC3, GC12")
        plt.plot(gc3, intercept + slope*gc3, "r", label = "Fitted line.\nb =" + str(round(slope, 2)) 
                 +", p = "+ str('{:0.3e}'.format(pvalue)))
    
        plt.xlabel("GC\u2083", fontsize = 30)
        plt.ylabel("GC\u2081"+"\u2082", fontsize = 30)
        plt.xlim([20, 57])
        plt.ylim([40, 53])
        plt.xticks(fontsize = 20)
        plt.yticks(fontsize = 20)
        plt.legend(fontsize = 25)
    
        if virus_genus != "Alfa":
            plt.title("Género $\it{%s}$" % virus_genus, fontsize = 35)
    
        else:
            plt.title("Género $\it{Alpha}$", fontsize = 35)
        
        number += 1
    
    plt.savefig(output_file)
    plt.show()

In [ ]:
genes = ['E1', 'E2', 'E4', 'E5', 'E6', 'E7', 'L1', 'L2']

In [ ]:
def plot_neutrality_genes(df_genes, output_file, genes):
    
    fig_size = plt.rcParams["figure.figsize"]
    fig_size[0] = 30
    fig_size[1] = 35
    plt.rcParams["figure.figsize"] = fig_size
    plt.subplots_adjust(wspace = 0.3, top = 0.93)

    number = 1

    #plt.suptitle("Género Alfa. Media de las secuencias por tipo", fontsize = 35)

    for gen in genes:
    
        gc3 = df_genes.loc[df_genes["Gen"].isin([gen])]["GC3"]
        gc12 = df_genes.loc[df_genes["Gen"].isin([gen])]["GC12"]
        
        slope, intercept, rvalue, pvalue, stderr = stats.linregress(gc3, gc12)

        plt.subplot(3,3, number)
        plt.plot(gc3, gc12, "o", label = "GC3, GC12")
        plt.plot(gc3, intercept + slope*gc3, "r", label = "Fitted line.\nb =" + str(round(slope, 2)) 
                 +", p = "+ str('{:0.3e}'.format(pvalue)))
    
        plt.xlabel("GC\u2083", fontsize = 30)
        plt.ylabel("GC\u2081"+"\u2082", fontsize = 30)
        plt.xlim([14, 72])
        plt.ylim([25, 65])
        plt.xticks(fontsize = 20)
        plt.yticks(fontsize = 20)
        plt.legend(fontsize = 25)
        plt.title("Gen "+gen, fontsize = 35)
        
        number += 1
    
    plt.savefig(output_file)
    plt.show()

In [ ]:
def medias_dataframe_gc(dataframe):
    
    means = pd.DataFrame()
    standard_error = pd.DataFrame()
    
    for genero in dataframe["Genero"].unique():
        
        mean_results = pd.DataFrame.mean(dataframe[dataframe["Genero"].isin([genero])], 0)
        
        mean_results = mean_results.to_frame()
        mean_results = mean_results.rename(columns = {0: genero})
        mean_results = mean_results.drop(mean_results.index[0])
        
        mean_results = mean_results.transpose()
        
        means = pd.concat([means, mean_results], axis = 0)
        standard_error_total_gc = stats.sem(dataframe["Total_GC"][dataframe["Genero"].isin([genero])])
        standard_error_gc12 = stats.sem(dataframe["GC12"][dataframe["Genero"].isin([genero])])
        standard_error_gc3 = stats.sem(dataframe["GC3"][dataframe["Genero"].isin([genero])])

        std_err = pd.DataFrame({"Total_GC" : [standard_error_total_gc], "GC12" : [standard_error_gc12], 
                      "GC3" : [standard_error_gc3]}, index = [genero])
        
        standard_error = pd.concat([standard_error, std_err], axis = 0)
    
    means = means.drop(columns = ["Tipo", "Total_GC_desv", "GC12_desv", "GC3_desv"])

    return means, standard_error

In [1]:
def medias_dataframe_gc_genes(dataframe_genes):
    
    means = pd.DataFrame()
    standard_error = pd.DataFrame()
    
    for genero in dataframe_genes["Genero"].unique():
        
        data_genero = dataframe_genes[dataframe_genes["Genero"].isin([genero])]
        
        for gen in genes:
            
            if gen in data_genero["Gen"].unique():
            
                mean_results = pd.DataFrame.mean(data_genero[data_genero["Gen"].isin([gen])], 0)
                
                mean_results = mean_results.to_frame()
                mean_results = mean_results.rename(columns = {0: gen})
                mean_results = mean_results.drop(mean_results.index[0])
        
                mean_results = mean_results.transpose()
                
                means = pd.concat([means, mean_results], axis = 0)
            
                standard_error_total_gc = stats.sem(dataframe_genes["Total_GC"][dataframe_genes["Genero"]\
                                                                                .isin([genero])]
                                                    [dataframe_genes["Gen"].isin([gen])])
                standard_error_gc12 = stats.sem(dataframe_genes["GC12"][dataframe_genes["Genero"]\
                                                                        .isin([genero])]
                                               [dataframe_genes["Gen"].isin([gen])])
                standard_error_gc3 = stats.sem(dataframe_genes["GC3"][dataframe_genes["Genero"]\
                                                                      .isin([genero])]
                                              [dataframe_genes["Gen"].isin([gen])])

                std_err = pd.DataFrame({"Total_GC" : [standard_error_total_gc], "GC12" : 
                                        [standard_error_gc12], 
                          "GC3" : [standard_error_gc3]}, index = [genero+"_"+gen])
        
                standard_error = pd.concat([standard_error, std_err], axis = 0)
        
    means = means.drop(["Tipo", "Total_GC_desv", "GC12_desv", "GC3_desv"], axis = 1)
    
    return means, standard_error

In [ ]:
def plot_gc_genome(df_gc_medias_1, df_gc_std_1, output_file):
    
    colores = plt.cm.get_cmap("YlGnBu")
    fig_size = plt.rcParams["figure.figsize"]
    fig_size[0] = 20
    fig_size[1] = 10
    plt.rcParams["figure.figsize"] = fig_size
    
    position = ["Alfa", "Beta", "Gamma"]
    x = np.arange(3)
    y = [40, 70, 120]
    barwidth = 0.20
    
    i = 0
#    plt.title(titulo_1, fontsize = 35)
        
    for gc_value in df_gc_medias_1.columns:
        
        plt.bar(x + (barwidth + 0.01) * i, df_gc_medias_1[gc_value], width = barwidth, 
               label = gc_value, color = colores(y[i]),
               yerr = df_gc_std_1[gc_value])
        plt.legend(fontsize = 25, loc = "lower right")
        
        i += 1
    plt.xticks([r + barwidth for r in range(len(x))], list((df_gc_medias_1.transpose()).columns),
              fontsize = 25) 
    plt.yticks(fontsize = 25)
    plt.xlabel("Género", fontsize = 23)
    plt.ylabel("Contenido G+C", fontsize = 23)

    plt.savefig(output_file)
    plt.show()

In [ ]:
def plot_gc_genes(df_medias, df_std, output_file):
    
    colores = plt.cm.get_cmap("YlGnBu")
    fig_size = plt.rcParams["figure.figsize"]
    fig_size[0] = 30
    fig_size[1] = 10
    plt.rcParams["figure.figsize"] = fig_size
    plt.subplots_adjust(hspace = 0.25)

    barwidth = 0.20
    x = np.arange(len(df_medias))
    y = [40, 70, 120]
    
    #plt.title(titulo, fontsize = 35)

    i = 0
    for column in list(df_medias.columns):
        plt.bar(x + (i * barwidth), df_medias[column], width = barwidth,
               label = column, color = colores(y[i]),
               yerr = df_std[column])

        i += 1
        
        
    plt.xticks([r + barwidth for r in range(len(x))], list((df_medias.transpose()).columns),
              fontsize = 25) 
    plt.yticks(fontsize = 25)
    plt.xlabel("Gen", fontsize = 23)
    plt.ylabel("Contenido G+C", fontsize = 23)
    plt.legend(fontsize = 25)

    plt.savefig(output_file)
    
    plt.show()